In [2]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
load_dotenv()


True

In [5]:
api_key=os.getenv("AZURE_OPENAI_API_KEY")
api_base=os.getenv("AZURE_OPENAI_API_BASE")
api_version=os.getenv("AZURE_OPENAI_API_VERSION")
deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

In [7]:
client = AzureOpenAI(
    api_key=api_key,
    azure_endpoint=api_base,
    api_version=api_version,
)

In [25]:
import requests

url = "https://weather-api167.p.rapidapi.com/api/weather/forecast"

#querystring = {"place":"London,GB","cnt":"3","units":"standard","type":"three_hour","mode":"json","lang":"en"}
def get_weather_forecast(place):
    
	headers = {
		"x-rapidapi-key": "80d7af33d9msh609e1f9303a6028p168c98jsn7a62ffe84df7",
		"x-rapidapi-host": "weather-api167.p.rapidapi.com",
		"Accept": "application/json"
	}
	querystring = {"place": place, "cnt": "3", "units": "standard", "type": "three_hour", "mode": "json", "lang": "en"}

	response = requests.get(url, headers=headers, params=querystring)

	return response.json()


In [30]:
weather_result=get_weather_forecast("trichy")

In [31]:
weather_result

{'cod': '200',
 'message': 0,
 'cnt': 3,
 'list': [{'dt': 1751209200,
   'summery': 'light rain, 306.17K, feels like 309.2K, winds 3.28 meter/sec from West, 49% humidity',
   'main': {'temprature': 306.17,
    'temprature_feels_like': 309.2,
    'temprature_min': 304.23,
    'temprature_max': 306.17,
    'temprature_unit': 'K',
    'pressure': 1007,
    'sea_level_pressure': 1007,
    'ground_level_pressure': 997,
    'pressure_unit': 'hPa',
    'humidity': 49,
    'humidity_unit': '%',
    'temp_kf': 1.94},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': 'https://openweathermap.org/img/wn/10n@2x.png'}],
   'clouds': {'cloudiness': 40, 'unit': '%'},
   'wind': {'speed': 3.28,
    'degrees': 268,
    'direction': 'West',
    'gust_speed': 6.8,
    'speed_unit': 'meter/sec'},
   'visibility_distance': 10000,
   'visibility_unit': 'm',
   'probability_of_precipitation': 57.99999999999999,
   'probability_of_precipitation_unit': '%',
   'rain'

In [36]:

def call_llm(query,function):
    
    result=client.chat.completions.create(
        model=deployment_name,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant that provides weather information."
            },
            {
                "role": "user",
                "content": {query}
            }
        ],
        max_tokens=2048,
        tools=function
        ).choices[0].message.content.strip()
    
    return result

## Defining function 

In [33]:
function=[
    {
        "name": "get_weather_forecast",
        "description": "Get the weather forecast for a given place.",
        "parameters": {
            "type": "object",
            "properties": {
                "place": {
                    "type": "string",
                    "description": "The place for which to get the weather forecast."
                }
            },
            "required": ["place"]
        }
    }
]

In [37]:
call_llm("What is the weather forecast for Trichy?",function)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'messages[1].content[0]': expected an object, but got a string instead.", 'type': 'invalid_request_error', 'param': 'messages[1].content[0]', 'code': 'invalid_type'}}